In [1]:
!pip -q install -U transformers datasets accelerate evaluate sentencepiece sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 74.5 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 35.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━

In [2]:
#setup and imports

In [3]:
# !pip -q install sentencepiece sacrebleu datasets kagglehub

import os, math, random, time
import numpy as np
import torch, torch.nn as nn
from typing import List, Tuple
from datasets import Dataset, DatasetDict
import sentencepiece as spm
import sacrebleu

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); 
if DEVICE=="cuda": torch.cuda.manual_seed_all(SEED)

print("Device:", DEVICE)

Device: cuda


In [4]:
# Expecting train.csv, valid.csv (and optionally test.csv)
import kagglehub
from kagglehub import KaggleDatasetAdapter


# Set the path to the file you'd like to load
file_path = "test.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "arefehrajabian/persian-pos-ner",
  file_path,

)

print("First 5 records:", df.head())

/tmp/ipykernel_36/1068884255.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:                                              persian  \
0                   و به زیر لیوانی کاغذی من نگاه کن   
1                           تو برو یه همچین کاری بکن   
2                                              پذیرش   
3  مدیامکس اخبار قره باغ کوهستانی وزیر خارجه ارمن...   
4  صد خانه در آن واحد متعارف و ترسناک در زیر آسما...   

                                             english  \
0         and look down at my embossed paper coaster   
1                  you go and do something like this   
2                                            adopcje   
3  mediamax news nagorno karabakh armenian fm bak...   
4  a hundred houses at once conventional and grot...   

                                         persian_pos  \
0       CCONJ ADP NOUN,EZ NOUN ADJ,EZ PRON NOUN VERB   
1                       PRON VERB NUM NOUN NOUN VERB   
2                                               NOUN   
3  NOUN NOUN,EZ NOUN,EZ ADJ,EZ NOUN,EZ ADJ,EZ NOU...   
4  NUM NOUN ADP NOUN,EZ NOUN,

In [5]:
# !pip -q install kagglehub datasets

import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter
from datasets import Dataset, DatasetDict

SLUG = "arefehrajabian/persian-pos-ner"   # your dataset

# Adjust these filenames if your split names differ
TRAIN_FILE = "train.csv"
VALID_FILE = "validation.csv"      # change to "validation.csv" or "dev.csv" if needed
TEST_FILE  = "test.csv"       # optional

# Map the dataset's column names -> our standard 'src'/'tgt'
SRC_COL_IN = "english"        # <- source (EN)
TGT_COL_IN = "persian"        # <- target (FA)

def safe_load_csv(slug, file_path):
    try:
        return kagglehub.load_dataset(
            KaggleDatasetAdapter.PANDAS,
            slug,
            file_path,
        )
    except Exception as e:
        print(f"Skipping '{file_path}' — not found or not a CSV. ({e})")
        return None

dfs = {
    "train":      safe_load_csv(SLUG, TRAIN_FILE),
    "validation": safe_load_csv(SLUG, VALID_FILE),
    "test":       safe_load_csv(SLUG, TEST_FILE),
}
dfs = {k: v for k, v in dfs.items() if v is not None}

for split, df in dfs.items():
    print(f"{split}: shape={df.shape}, cols={list(df.columns)}")

def standardize(df, src_in=SRC_COL_IN, tgt_in=TGT_COL_IN):
    if src_in not in df.columns or tgt_in not in df.columns:
        raise ValueError(f"Expected columns '{src_in}' and '{tgt_in}', found {list(df.columns)}")
    out = df[[src_in, tgt_in]].rename(columns={src_in: "src", tgt_in: "tgt"})
    out = out.dropna(subset=["src", "tgt"]).drop_duplicates().reset_index(drop=True)
    return out

dfs_std = {split: standardize(df) for split, df in dfs.items()}

# Convert to HF datasets
raw = DatasetDict({split: Dataset.from_pandas(df.reset_index(drop=True)) for split, df in dfs_std.items()})

# Keep the rest of your pipeline expecting these names:
SRC_COL, TGT_COL = "src", "tgt"
TEST_REF_COL = "ref"  # only relevant if your test file has references

print("\nFinal splits:")
for split in raw.keys():
    print(split, "→", raw[split].num_rows, raw[split].column_names)


/tmp/ipykernel_36/2724337234.py:21: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  return kagglehub.load_dataset(
/tmp/ipykernel_36/2724337234.py:21: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  return kagglehub.load_dataset(
/tmp/ipykernel_36/2724337234.py:21: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  return kagglehub.load_dataset(


train: shape=(136396, 4), cols=['persian', 'english', 'persian_pos', 'fa_ner_seq']
validation: shape=(5990, 4), cols=['persian', 'english', 'persian_pos', 'fa_ner_seq']
test: shape=(5991, 4), cols=['persian', 'english', 'persian_pos', 'fa_ner_seq']

Final splits:
train → 136158 ['src', 'tgt']
validation → 5989 ['src', 'tgt']
test → 5991 ['src', 'tgt']


**1) Train a shared SentencePiece tokenizer (BPE/Unigram)**

In [6]:
# Collect text for tokenizer training (subset if dataset is huge)
def iter_text(ds, col):
    for x in ds[col]:
        if x is None: 
            continue
        s = str(x).strip()
        if s: 
            yield s

# Feed both EN+FA so we get a shared vocab
train_src = list(iter_text(raw["train"], "src"))
train_tgt = list(iter_text(raw["train"], "tgt"))
tok_train_path = "/kaggle/working/spm_corpus.txt"
with open(tok_train_path, "w", encoding="utf-8") as f:
    for s in train_src + train_tgt:
        f.write(s.replace("\n"," ") + "\n")

VOCAB_SIZE = 16000
SPM_PREFIX = "/kaggle/working/spm_en_fa"
spm.SentencePieceTrainer.Train(
    input=tok_train_path, 
    model_prefix=SPM_PREFIX, 
    vocab_size=VOCAB_SIZE,
    model_type="unigram",             # or "bpe"
    character_coverage=1.0,
    pad_id=0, unk_id=1, bos_id=2, eos_id=3,  # reserve ids
    input_sentence_size=2000000,      # subsample if very large
    shuffle_input_sentence=True
)

sp = spm.SentencePieceProcessor()
sp.Load(f"{SPM_PREFIX}.model")

PAD, UNK, BOS, EOS = 0, 1, 2, 3
print("SentencePiece loaded. Example pieces:", sp.EncodeAsPieces("Hello Tehran!"))


SentencePiece loaded. Example pieces: ['▁', 'H', 'el', 'lo', '▁', 'T', 'e', 'hran', '!']


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /kaggle/working/spm_corpus.txt
  input_format: 
  model_prefix: /kaggle/working/spm_en_fa
  model_type: UNIGRAM
  vocab_size: 16000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 2000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id: 2
  eos_id: 3
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  en

**2) Encode datasets → PyTorch-friendly Datasets + DataLoaders**

In [7]:
from typing import List, Tuple
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch


MAX_SRC_LEN = 160
MAX_TGT_LEN = 160
BATCH_SIZE  = 64   # adjust if OOM

def encode_example(src: str, tgt: str) -> Tuple[List[int], List[int]]:
    src_ids = sp.EncodeAsIds(str(src))[:MAX_SRC_LEN-1] + [EOS]
    tgt_ids = [BOS] + sp.EncodeAsIds(str(tgt))[:MAX_TGT_LEN-2] + [EOS]
    return src_ids, tgt_ids

class MTEncodedDataset(Dataset):
    def __init__(self, hf_split):
        self.src = hf_split["src"]
        self.tgt = hf_split["tgt"]
    def __len__(self): return len(self.src)
    def __getitem__(self, i):
        return encode_example(self.src[i], self.tgt[i])

def collate(batch):
    # batch: list of (src_ids, tgt_ids)
    src_seqs, tgt_seqs = zip(*batch)
    src_lens = torch.tensor([len(s) for s in src_seqs])
    tgt_lens = torch.tensor([len(t) for t in tgt_seqs])

    def pad_to_max(seqs, pad=PAD):
        maxlen = max(len(s) for s in seqs)
        out = torch.full((maxlen, len(seqs)), pad, dtype=torch.long)
        for i, s in enumerate(seqs):
            out[:len(s), i] = torch.tensor(s, dtype=torch.long)
        return out

    src_pad = pad_to_max(src_seqs, PAD)  # (T,B)
    tgt_pad = pad_to_max(tgt_seqs, PAD)  # (T,B)
    return src_pad, src_lens, tgt_pad, tgt_lens

ds_train = MTEncodedDataset(raw["train"])
ds_valid = MTEncodedDataset(raw["validation"])
dl_train = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate, drop_last=False)
dl_valid = DataLoader(ds_valid, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate, drop_last=False)
len(ds_train), len(ds_valid)


(136158, 5989)

**Build the LSTM seq2seq with additive attention**

In [8]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=layers, bidirectional=True,
                           dropout=0 if layers==1 else dropout)
        self.dropout = nn.Dropout(dropout)
        self.hid_dim = hid_dim
        self.layers = layers

    def forward(self, src, src_lens):
        # src: (T,B)
        emb = self.dropout(self.emb(src))  # (T,B,E)
        packed = nn.utils.rnn.pack_padded_sequence(emb, src_lens.cpu(), enforce_sorted=False)
        out, (h, c) = self.rnn(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(out)  # (T,B,2H)
        # reshape h,c: (2*L,B,H) -> (L,B,2H)
        h = torch.cat([h[0::2], h[1::2]], dim=2)
        c = torch.cat([c[0::2], c[1::2]], dim=2)
        return out, (h, c)  # out for attention

# ---------- 1) Attention patched for AMP (no fp16 overflow) ----------
import torch, torch.nn as nn

class AdditiveAttention(nn.Module):
    def __init__(self, enc_dim, dec_dim, attn_dim=256):
        super().__init__()
        self.W = nn.Linear(enc_dim + dec_dim, attn_dim)
        self.v = nn.Linear(attn_dim, 1, bias=False)

    def forward(self, dec_hidden, enc_outs, src_mask):
        # dec_hidden: (B,Hd), enc_outs: (T,B,He), src_mask: (B,T) bool
        T, B, He = enc_outs.size()
        orig_dtype = enc_outs.dtype

        # build energy in original dtype
        dec = dec_hidden.unsqueeze(0).expand(T, B, -1)                # (T,B,Hd)
        energy = torch.tanh(self.W(torch.cat([dec, enc_outs], dim=2)))# (T,B,A)

        # scores/softmax in float32 to avoid fp16 overflow
        scores = self.v(energy).squeeze(2).transpose(0,1).float()     # (B,T), fp32

        # mask safely (fp32)
        fill_val = -1e4 if scores.dtype == torch.float16 else -1e9
        scores = scores.masked_fill(~src_mask, fill_val)

        attn = torch.softmax(scores, dim=1)                            # (B,T), fp32
        # context in fp32, then cast back to original dtype
        context = torch.bmm(attn.unsqueeze(1), enc_outs.transpose(0,1).float()).squeeze(1)  # (B,He) fp32
        context = context.to(orig_dtype)
        return context, attn




class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, enc_dim, dec_hid, layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD)
        self.rnn = nn.LSTM(emb_dim + enc_dim, dec_hid, num_layers=layers,
                           dropout=0 if layers==1 else dropout)
        self.attn = AdditiveAttention(enc_dim, dec_hid)
        self.fc = nn.Linear(dec_hid + enc_dim + emb_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, y_prev, hidden, cell, enc_outs, src_mask):
        emb = self.dropout(self.emb(y_prev)).unsqueeze(0)   # (1,B,E)
        dec_hidden = hidden[-1]                             # (B,Hd)
        context, _ = self.attn(dec_hidden, enc_outs, src_mask)  # (B,He)
        rnn_in = torch.cat([emb, context.unsqueeze(0)], dim=2)  # (1,B,E+He)
        out, (h,c) = self.rnn(rnn_in, (hidden, cell))           # out: (1,B,Hd)
        out = out.squeeze(0)                                    # (B,Hd)
        logits = self.fc(torch.cat([out, context, emb.squeeze(0)], dim=1))  # (B,V)
        return logits, (h,c)

class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, emb=256, enc_h=384, dec_h=512, enc_layers=1, dec_layers=1, dropout=0.2):
        super().__init__()
        self.enc = Encoder(vocab_size, emb, enc_h, enc_layers, dropout)
        self.dec = Decoder(vocab_size, emb, enc_h*2, dec_h, dec_layers, dropout)
        # (optional) tie decoder output to embeddings for smaller models
        # self.dec.fc.weight = self.dec.emb.weight

    def make_src_mask(self, src):
        return (src.transpose(0,1) != PAD)   # (B,T)

    def forward(self, src, src_lens, tgt, teacher_forcing=0.8):
        enc_outs, (h,c) = self.enc(src, src_lens)            # enc_outs: (T,B,2Henc)
        src_mask = self.make_src_mask(src)                   # (B,T)
        B = src.size(1)
        y = tgt[0]  # BOS (assuming first token is BOS)
        logits_seq = []
        # project encoder state to decoder dims if needed (here we keep simple: zeros)
        hidden = torch.zeros(self.dec.rnn.num_layers, B, self.dec.rnn.hidden_size, device=src.device)
        cell   = torch.zeros(self.dec.rnn.num_layers, B, self.dec.rnn.hidden_size, device=src.device)
        for t in range(1, tgt.size(0)):
            logits, (hidden, cell) = self.dec(y, hidden, cell, enc_outs, src_mask)
            logits_seq.append(logits.unsqueeze(0))
            y = tgt[t] if (random.random()<teacher_forcing) else logits.argmax(dim=1)
        return torch.cat(logits_seq, dim=0)   # (T-1,B,V)

    @torch.no_grad()
    def generate(self, src, src_lens, max_len=160, beam_size=1):
        self.eval()
        enc_outs, (h,c) = self.enc(src, src_lens)
        src_mask = self.make_src_mask(src)
        B = src.size(1)

        # Greedy only (beam=1) for simplicity here
        y = torch.full((B,), BOS, dtype=torch.long, device=src.device)
        hidden = torch.zeros(self.dec.rnn.num_layers, B, self.dec.rnn.hidden_size, device=src.device)
        cell   = torch.zeros(self.dec.rnn.num_layers, B, self.dec.rnn.hidden_size, device=src.device)
        outs = []
        for _ in range(max_len):
            logits, (hidden, cell) = self.dec(y, hidden, cell, enc_outs, src_mask)
            y = logits.argmax(dim=1)
            outs.append(y)
            if (y==EOS).all(): break
        return torch.stack(outs, 0)  # (T,B)


In [9]:
import torch
import torch.nn as nn

# you already have `sp` and `Seq2Seq` defined/imported

# --- hyperparameters (set to whatever you want)
EMB_DIM = 512
ENC_H   = 512
DEC_H   = 512
LR      = 3e-4
LABEL_SMOOTH = 0.1

# --- special token ids
pad_id = getattr(sp, "PieceToId")("<pad>") if hasattr(sp, "PieceToId") else -1
PAD = pad_id if pad_id != -1 else 0  # fallback if your model has no <pad> piece

# --- device
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


**train the model**

In [10]:
# VOCAB_SIZE = sp.GetPieceSize()
# model = Seq2Seq(VOCAB_SIZE, emb=EMB_DIM, enc_h=ENC_H, dec_h=DEC_H, enc_layers=1, dec_layers=1, dropout=0.2).to(DEVICE)
# optim = torch.optim.AdamW(model.parameters(), lr=LR)
# try:
#     crit = nn.CrossEntropyLoss(ignore_index=PAD, label_smoothing=LABEL_SMOOTH)
# except TypeError:
#     crit = nn.CrossEntropyLoss(ignore_index=PAD)

**first try(interrupt in second epoch)**

In [ ]:
# ===== Fix & full training block (safe, self-contained) =====
import math, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple

# --- sanity checks (raw + sp must exist) ---
assert 'train' in raw and 'validation' in raw, "Expected raw['train'] and raw['validation']"
for split in ['train','validation']:
    assert 'src' in raw[split].column_names and 'tgt' in raw[split].column_names, "raw must have 'src' and 'tgt'"
assert 'EncodeAsIds' in dir(sp), "SentencePiece processor `sp` not loaded"

# --- special tokens (match your SPM training) ---
PAD, UNK, BOS, EOS = 0, 1, 2, 3

# --- hyperparams (edit as you like) ---
MAX_SRC_LEN = 160
MAX_TGT_LEN = 160
BATCH_SIZE  = 64
EMB_DIM = 256
ENC_H   = 384
DEC_H   = 512
EPOCHS  = 5
LR      = 3e-4
CLIP    = 1.0
LABEL_SMOOTH = 0.1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- dataset & collate that ALWAYS returns a 4-tuple ---
def _encode_example(src: str, tgt: str) -> Tuple[list, list]:
    src_ids = sp.EncodeAsIds(str(src))[:MAX_SRC_LEN-1] + [EOS]
    tgt_ids = [BOS] + sp.EncodeAsIds(str(tgt))[:MAX_TGT_LEN-2] + [EOS]
    return src_ids, tgt_ids

class MTEncodedDataset(Dataset):
    def __init__(self, hf_split):
        self.src = hf_split["src"]
        self.tgt = hf_split["tgt"]
    def __len__(self): return len(self.src)
    def __getitem__(self, i):
        return _encode_example(self.src[i], self.tgt[i])

def collate(batch):
    # batch: list of (src_ids, tgt_ids)
    src_seqs, tgt_seqs = zip(*batch)
    src_lens = torch.tensor([len(s) for s in src_seqs], dtype=torch.long)
    tgt_lens = torch.tensor([len(t) for t in tgt_seqs], dtype=torch.long)
    def pad_to_max(seqs, pad=PAD):
        T = max(len(s) for s in seqs)
        B = len(seqs)
        out = torch.full((T, B), pad, dtype=torch.long)
        for i, s in enumerate(seqs):
            out[:len(s), i] = torch.tensor(s, dtype=torch.long)
        return out
    src_pad = pad_to_max(src_seqs, PAD)  # (T,B)
    tgt_pad = pad_to_max(tgt_seqs, PAD)  # (T,B)
    return src_pad, src_lens, tgt_pad, tgt_lens  # <<< 4-tuple

# --- build loaders fresh (guarantees tuple batches) ---
ds_train = MTEncodedDataset(raw["train"])
ds_valid = MTEncodedDataset(raw["validation"])
dl_train = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate, drop_last=False)
dl_valid = DataLoader(ds_valid, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate, drop_last=False)

# --- quick diagnostic (optional) ---
first = next(iter(dl_train))
assert isinstance(first, (list, tuple)) and len(first) == 4, "Batch must be a 4-tuple (src, src_lens, tgt, tgt_lens)"

# --- model, optim, loss ---
VOCAB_SIZE = sp.GetPieceSize()
model = Seq2Seq(VOCAB_SIZE, emb=EMB_DIM, enc_h=ENC_H, dec_h=DEC_H, enc_layers=1, dec_layers=1, dropout=0.2).to(DEVICE)
optim = torch.optim.AdamW(model.parameters(), lr=LR)
try:
    crit = nn.CrossEntropyLoss(ignore_index=PAD, label_smoothing=LABEL_SMOOTH)
except TypeError:
    crit = nn.CrossEntropyLoss(ignore_index=PAD)

# --- robust unpack + epoch runner ---
def _unpack_batch(batch):
    # batch is guaranteed a 4-tuple from our collate; keep flexible anyway
    if isinstance(batch, (list, tuple)) and len(batch) >= 3:
        src, src_lens, tgt = batch[0], batch[1], batch[2]
        return src, src_lens, tgt
    elif isinstance(batch, dict):
        src = batch.get("src"); src_lens = batch.get("src_lens")
        tgt = batch.get("tgt") or batch.get("labels")
        if src is None or src_lens is None or tgt is None:
            raise ValueError(f"Unexpected batch dict keys: {list(batch.keys())}")
        return src, src_lens, tgt
    else:
        raise TypeError(f"Unsupported batch type: {type(batch)}")

def run_epoch(dataloader, train=True):
    model.train() if train else model.eval()
    total_loss, total_tokens = 0.0, 0
    for batch in dataloader:
        src, src_lens, tgt = _unpack_batch(batch)
        src, src_lens, tgt = src.to(DEVICE), src_lens.to(DEVICE), tgt.to(DEVICE)

        if train:
            logits = model(src, src_lens, tgt, teacher_forcing=0.8)   # (T-1,B,V)
        else:
            with torch.no_grad():
                logits = model(src, src_lens, tgt, teacher_forcing=1.0)

        gold = tgt[1:].contiguous().view(-1)  # shift
        loss = crit(logits.view(-1, logits.size(-1)), gold)

        if train:
            optim.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), CLIP)
            optim.step()

        ntoks = (gold != PAD).sum().item()
        total_loss += float(loss) * ntoks
        total_tokens += ntoks

    return total_loss / max(total_tokens, 1)

# --- train loop (saves best by val loss) ---
best_val = float("inf")
best_path = "/kaggle/working/seq2seq_best.pt"
for epoch in range(1, EPOCHS+1):
    tr_loss = run_epoch(dl_train, train=True)
    val_loss = run_epoch(dl_valid, train=False)
    print(f"Epoch {epoch}: train_ppl={math.exp(tr_loss):.2f}  val_ppl={math.exp(val_loss):.2f}")
    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), best_path)
        print("Saved best checkpoint ->", best_path)

print("Done.")


Epoch 1: train_ppl=463.23  val_ppl=183.27
Saved best checkpoint -> /kaggle/working/seq2seq_best.pt


# continue training

In [ ]:
# import os, time, math, torch, torch.nn as nn

# best_path = "/kaggle/working/seq2seq_best.pt"
# last_path = "/kaggle/working/seq2seq_last.pt"

# # ---------- Encoder/Decoder with fp32-safe attention ----------
# class Encoder(nn.Module):
#     def __init__(self, vocab_size, emb_dim, hid_dim, layers=1, dropout=0.2):
#         super().__init__()
#         self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD)
#         self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=layers, bidirectional=True,
#                            dropout=0 if layers==1 else dropout)
#         self.dropout = nn.Dropout(dropout)
#     def forward(self, src, src_lens):
#         emb = self.dropout(self.emb(src))                          # (T,B,E)
#         packed = nn.utils.rnn.pack_padded_sequence(emb, src_lens.cpu(), enforce_sorted=False)
#         out,(h,c) = self.rnn(packed)
#         out,_ = nn.utils.rnn.pad_packed_sequence(out)              # (T,B,2H)
#         h = torch.cat([h[0::2], h[1::2]], dim=2)                   # (L,B,2H)
#         c = torch.cat([c[0::2], c[1::2]], dim=2)
#         return out,(h,c)

# class AdditiveAttention(nn.Module):
#     def __init__(self, enc_dim, dec_dim, attn_dim=256):
#         super().__init__()
#         self.W = nn.Linear(enc_dim + dec_dim, attn_dim)
#         self.v = nn.Linear(attn_dim, 1, bias=False)
#     def forward(self, dec_hidden, enc_outs, src_mask):
#         # enc_outs: (T,B,He)  dec_hidden: (B,Hd)  src_mask: (B,T) bool
#         T,B,He = enc_outs.size()
#         orig_dtype = enc_outs.dtype
#         dec = dec_hidden.unsqueeze(0).expand(T,B,-1)               # (T,B,Hd)
#         energy = torch.tanh(self.W(torch.cat([dec, enc_outs], dim=2)))
#         scores = self.v(energy).squeeze(2).transpose(0,1).float()  # (B,T), fp32
#         scores = scores.masked_fill(~src_mask, -1e9)               # mask in fp32
#         attn   = torch.softmax(scores, dim=1)                      # (B,T), fp32
#         context = torch.bmm(attn.unsqueeze(1), enc_outs.transpose(0,1).float()).squeeze(1)  # (B,He), fp32
#         return context.to(orig_dtype), attn

# class Decoder(nn.Module):
#     def __init__(self, vocab_size, emb_dim, enc_dim, dec_hid, layers=1, dropout=0.2):
#         super().__init__()
#         self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD)
#         self.rnn = nn.LSTM(emb_dim + enc_dim, dec_hid, num_layers=layers,
#                            dropout=0 if layers==1 else dropout)
#         self.attn = AdditiveAttention(enc_dim, dec_hid)
#         self.fc = nn.Linear(dec_hid + enc_dim + emb_dim, vocab_size)
#         self.dropout = nn.Dropout(dropout)
#     def forward(self, y_prev, hidden, cell, enc_outs, src_mask):
#         emb = self.dropout(self.emb(y_prev)).unsqueeze(0)          # (1,B,E)
#         dec_hidden = hidden[-1]                                     # (B,Hd)
#         context,_ = self.attn(dec_hidden, enc_outs, src_mask)       # (B,He)
#         rnn_in = torch.cat([emb, context.unsqueeze(0)], dim=2)      # (1,B,E+He)
#         out,(h,c) = self.rnn(rnn_in, (hidden, cell))                # out: (1,B,Hd)
#         out = out.squeeze(0)                                        # (B,Hd)
#         logits = self.fc(torch.cat([out, context, emb.squeeze(0)], dim=1))  # (B,V)
#         return logits,(h,c)

# class Seq2Seq(nn.Module):
#     def __init__(self, vocab_size, emb=256, enc_h=384, dec_h=512, enc_layers=1, dec_layers=1, dropout=0.2):
#         super().__init__()
#         self.enc = Encoder(vocab_size, emb, enc_h, enc_layers, dropout)
#         self.dec = Decoder(vocab_size, emb, enc_h*2, dec_h, dec_layers, dropout)
#     def make_src_mask(self, src):
#         # src: (T,B) -> mask: (B,T) bool
#         return (src.transpose(0,1) != PAD).bool()
#     def forward(self, src, src_lens, tgt, teacher_forcing=0.8):
#         enc_outs,(h,c) = self.enc(src, src_lens)
#         mask = self.make_src_mask(src)
#         B = src.size(1)
#         y = tgt[0]  # BOS
#         hidden = torch.zeros(self.dec.rnn.num_layers, B, self.dec.rnn.hidden_size, device=src.device)
#         cell   = torch.zeros(self.dec.rnn.num_layers, B, self.dec.rnn.hidden_size, device=src.device)
#         outs=[]
#         for t in range(1, tgt.size(0)):
#             logits,(hidden,cell) = self.dec(y, hidden, cell, enc_outs, mask)
#             outs.append(logits.unsqueeze(0))
#             y = tgt[t] if (torch.rand(1, device=src.device).item() < teacher_forcing) else logits.argmax(dim=1)
#         return torch.cat(outs, 0)
#     @torch.no_grad()
#     def generate(self, src, src_lens, max_len=160):
#         self.eval()
#         enc_outs,(h,c) = self.enc(src, src_lens)
#         mask = self.make_src_mask(src)
#         B = src.size(1)
#         y = torch.full((B,), BOS, dtype=torch.long, device=src.device)
#         hidden = torch.zeros(self.dec.rnn.num_layers, B, self.dec.rnn.hidden_size, device=src.device)
#         cell   = torch.zeros(self.dec.rnn.num_layers, B, self.dec.rnn.hidden_size, device=src.device)
#         outs=[]
#         for _ in range(max_len):
#             logits,(hidden,cell) = self.dec(y, hidden, cell, enc_outs, mask)
#             y = logits.argmax(dim=1)
#             outs.append(y)
#             if (y==EOS).all(): break
#         return torch.stack(outs,0)

# # ---------- rebuild model and RESUME ----------
# VOCAB_SIZE = sp.GetPieceSize()
# model = Seq2Seq(VOCAB_SIZE, emb=EMB_DIM, enc_h=ENC_H, dec_h=DEC_H).to(DEVICE)

# # (re)create optimizer; we'll load state if resuming from "last"
# optim = torch.optim.AdamW(model.parameters(), lr=LR)

# # new AMP API
# scaler = torch.amp.GradScaler('cuda', enabled=(DEVICE=='cuda'))

# start_epoch = 1
# if os.path.exists(last_path):
#     ckpt = torch.load(last_path, map_location=DEVICE)
#     model.load_state_dict(ckpt["model"])
#     try: optim.load_state_dict(ckpt["optim"])
#     except: pass
#     try: scaler.load_state_dict(ckpt["scaler"])
#     except: pass
#     start_epoch = ckpt.get("epoch", 0) + 1
#     print(f"[resume] Loaded LAST checkpoint at epoch {start_epoch-1}")
# elif os.path.exists(best_path):
#     model.load_state_dict(torch.load(best_path, map_location=DEVICE))
#     print("[resume] Loaded BEST checkpoint")
# else:
#     print("[resume] No checkpoint found; starting fresh")

# # ---------- safe epoch runner ----------
# def run_epoch(dataloader, train=True):
#     model.train() if train else model.eval()
#     total_loss, total_tokens = 0.0, 0
#     for src, src_lens, tgt, _ in dataloader:
#         src, src_lens, tgt = src.to(DEVICE), src_lens.to(DEVICE), tgt.to(DEVICE)
#         if train:
#             with torch.amp.autocast('cuda', enabled=(DEVICE=='cuda')):
#                 logits = model(src, src_lens, tgt, teacher_forcing=0.8)
#                 gold   = tgt[1:].contiguous().view(-1)
#                 loss   = crit(logits.view(-1, logits.size(-1)), gold)
#             optim.zero_grad(set_to_none=True)
#             scaler.scale(loss).backward()
#             scaler.unscale_(optim)
#             nn.utils.clip_grad_norm_(model.parameters(), CLIP)
#             scaler.step(optim); scaler.update()
#         else:
#             with torch.no_grad(), torch.amp.autocast('cuda', enabled=(DEVICE=='cuda')):
#                 logits = model(src, src_lens, tgt, teacher_forcing=1.0)
#                 gold   = tgt[1:].contiguous().view(-1)
#                 loss   = crit(logits.view(-1, logits.size(-1)), gold)
#         ntoks = (gold != PAD).sum().item()
#         total_loss += float(loss) * ntoks
#         total_tokens += ntoks
#     return total_loss / max(total_tokens, 1)

# # ---------- continue training ----------
# best_val = float("inf")
# for epoch in range(start_epoch, EPOCHS+1):
#     t0 = time.time()
#     tr_loss = run_epoch(dl_train, train=True)
#     val_loss = run_epoch(dl_valid, train=False)
#     dt = time.time() - t0
#     print(f"Epoch {epoch}: train_ppl={math.exp(tr_loss):.2f}  val_ppl={math.exp(val_loss):.2f}  time={dt/60:.1f}m")

#     torch.save({"epoch": epoch,
#                 "model": model.state_dict(),
#                 "optim": optim.state_dict(),
#                 "scaler": scaler.state_dict()}, last_path)

#     if val_loss < best_val:
#         best_val = val_loss
#         torch.save(model.state_dict(), best_path)
#         print("Continue saving best checkpoint ->", best_path)


In [ ]:
# resume_path = "/kaggle/working/seq2seq_last.pt"
# start_epoch = 1
# if os.path.exists(resume_path):
#     ckpt = torch.load(resume_path, map_location=DEVICE)
#     model.load_state_dict(ckpt["model"])
#     optim.load_state_dict(ckpt["optim"])
#     try:
#         scaler.load_state_dict(ckpt["scaler"])
#     except Exception:
#         pass
#     start_epoch = ckpt.get("epoch", 0) + 1
#     print(f"Resuming from epoch {start_epoch} (loaded {resume_path})")


**first try:**

In [ ]:
# reload best weights for inference
model.load_state_dict(torch.load("/kaggle/working/seq2seq_best.pt", map_location=DEVICE))
model.eval()

def encode_src_batch(texts):
    ids = [sp.EncodeAsIds(s)[:MAX_SRC_LEN-1] + [EOS] for s in texts]
    lens = torch.tensor([len(x) for x in ids])
    T = max(lens).item(); B = len(ids)
    pad = torch.full((T,B), PAD, dtype=torch.long)
    for i, seq in enumerate(ids):
        pad[:len(seq), i] = torch.tensor(seq)
    return pad.to(DEVICE), lens.to(DEVICE)

def decode_ids(ids):
    if EOS in ids: ids = ids[:ids.index(EOS)]
    if ids and ids[0]==BOS: ids = ids[1:]
    return sp.DecodeIds(ids)

@torch.no_grad()
def translate(texts, max_len=MAX_TGT_LEN):
    if isinstance(texts, str): texts = [texts]
    src, src_lens = encode_src_batch(texts)
    gen = model.generate(src, src_lens, max_len=max_len)
    return [decode_ids(gen[:,b].tolist()) for b in range(gen.size(1))]

print(translate(["Tehran is the capital of Iran.", "How are you today?"]))


In [ ]:
# reload best weights for inference
model.load_state_dict(torch.load("/kaggle/working/seq2seq_best.pt", map_location=DEVICE))
model.eval()

def encode_src_batch(texts):
    ids = [sp.EncodeAsIds(s)[:MAX_SRC_LEN-1] + [EOS] for s in texts]
    lens = torch.tensor([len(x) for x in ids])
    T = max(lens).item(); B = len(ids)
    pad = torch.full((T,B), PAD, dtype=torch.long)
    for i, seq in enumerate(ids):
        pad[:len(seq), i] = torch.tensor(seq)
    return pad.to(DEVICE), lens.to(DEVICE)

def decode_ids(ids):
    if EOS in ids: ids = ids[:ids.index(EOS)]
    if ids and ids[0]==BOS: ids = ids[1:]
    return sp.DecodeIds(ids)

@torch.no_grad()
def translate(texts, max_len=MAX_TGT_LEN):
    if isinstance(texts, str): texts = [texts]
    src, src_lens = encode_src_batch(texts)
    gen = model.generate(src, src_lens, max_len=max_len)
    return [decode_ids(gen[:,b].tolist()) for b in range(gen.size(1))]

print(translate(["Tehran is the capital of Iran.", "How are you today?"]))
print(translate(["this is arefe.", "How are you today?"]))

